In [31]:
import re

name_format = "page_0001_panel_0000_bubble_0001.jpg"

# Use regex to find all sequences of digits (including leading zeros)
numbers = re.findall(r'\d+', name_format)

print(numbers)

['0001', '0000', '0001']


# Change images name quick

In [ ]:
import os
import re

def rename_files(directory):
    # Change to the target directory
    os.chdir(directory)
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Use regex to find numbers and add leading zeros
        new_filename = re.sub(r'_(\d+)', lambda x: f'_{int(x.group(1)):04}', filename)
        
        # Rename the file
        os.rename(filename, new_filename)

# Specify the directory containing the image files
directory_path = 'path/to/your/directory'
rename_files(directory_path)

# Change chapter name to 01.ext format

In [ ]:
import os
import shutil

def rename_and_copy_images(images_folder, new_folder):
    original_image_path_list = []
    new_image_path_list = []

    # Create the new folder if it doesn't exist
    os.makedirs(new_folder, exist_ok=True)

    for idx, image_path in enumerate(os.listdir(images_folder)):
        original_path = os.path.join(images_folder, image_path)
        if os.path.isfile(original_path):  # Ensure it's a file
            original_image_path_list.append(image_path)
            original_ext = os.path.splitext(image_path)[1]
            new_name = f"{idx:03}{original_ext}"
            new_path = os.path.join(new_folder, new_name)
            shutil.copy2(original_path, new_path)
            new_image_path_list.append(new_name)

    print(f"original_image_path_list: {original_image_path_list}")
    print(f"new_image_path_list: {new_image_path_list}")

# Example usage
images_folder = '../../personal_data/One_Piece/raw'
new_folder = os.path.join(images_folder, 'renamed_images')
rename_and_copy_images(images_folder, new_folder)

original_image_path_list: ['One Piece - c002 (web) - p000 [Unknown].jpg', 'One Piece - c002 (web) - p001 [Unknown].jpg', 'One Piece - c002 (web) - p002 [Unknown].jpg', 'One Piece - c002 (web) - p003 [Unknown].jpg', 'One Piece - c002 (web) - p004 [Unknown].jpg', 'One Piece - c002 (web) - p005 [Unknown].jpg', 'One Piece - c002 (web) - p006 [Unknown].jpg', 'One Piece - c002 (web) - p007 [Unknown].jpg', 'One Piece - c002 (web) - p008 [Unknown].jpg', 'One Piece - c002 (web) - p009 [Unknown].jpg', 'One Piece - c002 (web) - p010 [Unknown].jpg', 'One Piece - c002 (web) - p011 [Unknown].jpg', 'One Piece - c002 (web) - p012 [Unknown].jpg', 'One Piece - c002 (web) - p013 [Unknown].jpg', 'One Piece - c002 (web) - p014 [Unknown].jpg', 'One Piece - c002 (web) - p015 [Unknown].jpg', 'One Piece - c002 (web) - p016 [Unknown].jpg', 'One Piece - c002 (web) - p017 [Unknown].jpg', 'One Piece - c002 (web) - p018 [Unknown].jpg', 'One Piece - c002 (web) - p019 [Unknown].jpg', 'One Piece - c002 (web) - p020 [U

# Generate name format

In [ ]:
import math

def generate_image_names(directory_path, buffer_number: int = 2):

    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif', '.webp', '.heif', '.ico')
    num_images = len([name for name in os.listdir(directory_path) if name.lower().endswith(image_extensions)])
     
    if num_images > 0:
        number_of_digit_for_name = math.ceil(math.log10(num_images+1)) + buffer_number
    else: 
        number_of_digit_for_name = buffer_number
    name_format = f"page{{:0{number_of_digit_for_name}}}_panel{{:0{number_of_digit_for_name}}}_bubble{{:0{number_of_digit_for_name}}}{{}}"
    return name_format

image_names = generate_image_names(buffer_number = 2)
print(image_names)

page{:04}_panel{:04}_bubble{:04}{}


# Single page

Testing to implement full page

In [ ]:
import json
import os
from PIL import Image, ImageDraw

def process_image_and_json(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    # Load the JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Extract the text coordinates and panel coordinates
    text_coords = [
    text for text, is_essential in zip(data["texts"], data["is_essential_text"]) if is_essential
]

    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        panel_index = 0
        # Create a draw object on the original image copy
        draw = ImageDraw.Draw(img)

        # Initialize the modified panel image path
        modified_panel_image_path = os.path.join(save_path, name_format)
        text_order = total_length_text

        # Save the modified panel image with the last computed bubble index
        img.save(modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg'))
        
        # Loop through each set of coordinates in reverse order
        for box_index, box in enumerate(reversed(text_coords)):
            x1, y1, x2, y2 = map(int, box)  # Ensures all values are integers

            # Initialize max color and max value
            max_color = (0, 0, 0)  # Start with black
            max_value = 0

            # Iterate through the box's pixels
            for y in range(y1, y2):
                for x in range(x1, x2):
                    # Get the pixel color using absolute coordinates
                    color = img.getpixel((x, y))
                    color_value = sum(color[:3])  # Sum RGB values

                    # Update max color if the current pixel's value is higher
                    if color_value > max_value:
                        max_value = color_value
                        max_color = color

            # Fill the rectangle with the max color
            draw.rectangle(box, fill=max_color)

            # Save modified image path for the current chat bubble
            text_order = total_length_text - (box_index + 1)
            modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg')

            # Instead of saving here, save it once at the end
            img.save(modified_image_path, format='JPEG')


    print(f"Processed and saved modified panel images for: {image_name_ext}")

def process_all_images_and_jsons(images_folder: str, json_folder: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}", nuke: bool = False):

    if nuke:
        image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}

        for filename in os.listdir(save_path):
            # Check if the file has an image extension
            if os.path.splitext(filename)[1].lower() in image_extensions:
                file_path = os.path.join(save_path, filename)
                os.remove(file_path)
        print(f"Deleted images")

    # Get all image and json file names
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    json_files = [f for f in os.listdir(json_folder) if f.lower().endswith('.json')]
    if len(image_files) != len(json_files):
        print("Number of images and json files do not match!")
    else:
        for image_file in image_files:
            # Construct full image path
            image_path = os.path.join(images_folder, image_file)

            # Corresponding json file name (assuming they have the same base name)
            json_file_name = os.path.splitext(image_file)[0] + '.json'
            if json_file_name in json_files:
                json_path = os.path.join(json_folder, json_file_name)

                # Process the image and json
                process_image_and_json(image_path, json_path, save_path, name_format)

# Example usage
images_folder = '../../personal_data/Ruri_Dragon/raw'  # Update this path
json_folder = '../../personal_data/Ruri_Dragon/json_results'  # Update this path
save_path = "./full_image"
name_format = "page_{:03}_panel_{:03}_bubble_{:03}{}"
nuke_option = True
process_all_images_and_jsons(images_folder, json_folder, save_path, name_format, nuke_option)

Deleted images
Processed and saved modified panel images for: 00.jpg
Processed and saved modified panel images for: 01.jpg
Processed and saved modified panel images for: 02.jpg
Processed and saved modified panel images for: 03.jpg
Processed and saved modified panel images for: 04.jpg
Processed and saved modified panel images for: 05.jpg
Processed and saved modified panel images for: 06.jpg


KeyboardInterrupt: 

# Works with panel

Function

Work with folder

In [ ]:
import json
import os
from PIL import Image, ImageDraw

def process_image_and_json(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    # Load the JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Extract the text coordinates and panel coordinates
    text_coords = [
    text for text, is_essential in zip(data["texts"], data["is_essential_text"]) if is_essential
]
    panel_coords = data["panels"]
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        # Create a copy of the image for drawing
        original_img = img.copy()

        # Loop through each panel
        for panel_index, panel in enumerate(panel_coords):
            # Create a panel-specific image by cropping the original image
            panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
            panel_image = original_img.crop(panel_box)

            # Create a draw object on the panel image copy
            draw = ImageDraw.Draw(panel_image)

            # Initialize the modified panel image path
            modified_panel_image_path = os.path.join(save_path, name_format)
            text_order = total_length_text

            # Save the modified panel image with the last computed bubble index
            panel_image.save(modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg'))
            
            # Loop through each set of coordinates in reverse order
            for box_index, box in enumerate(reversed(text_coords)):
                # box format: [x1, y1, x2, y2]
                # Calculate the center point of the chatbox
                x_center = (box[0] + box[2]) // 2
                y_center = (box[1] + box[3]) // 2

                # Check if the center point is inside the panel
                if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                    # Calculate the position of the box relative to the panel
                    relative_box = (
                        box[0] - panel[0],
                        box[1] - panel[1],
                        box[2] - panel[0],
                        box[3] - panel[1]
                    )

                    # Get the coordinates of the relative box and convert to integers
                    x1, y1, x2, y2 = map(int, relative_box)  # Ensures all values are integers

                    # Initialize max color and max value
                    max_color = (0, 0, 0)  # Start with black
                    max_value = 0

                    # Iterate through the box's pixels
                    for y in range(y1, y2):
                        for x in range(x1, x2):
                            # Get the pixel color using absolute coordinates
                            color = original_img.getpixel((x + panel[0], y + panel[1]))
                            color_value = sum(color[:3])  # Sum RGB values

                            # Update max color if the current pixel's value is higher
                            if color_value > max_value:
                                max_value = color_value
                                max_color = color

                    # Fill the rectangle with the max color
                    draw.rectangle(relative_box, fill=max_color)

                    # Save modified image path for the current chat bubble
                    text_order = total_length_text - (box_index + 1)
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg')

                    # Instead of saving here, save it once at the end
                    panel_image.save(modified_image_path, format='JPEG')


    print(f"Processed and saved modified panel images for: {image_name_ext}")

def process_all_images_and_jsons(images_folder: str, json_folder: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}", nuke: bool = False):

    if nuke:
        image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}

        for filename in os.listdir(save_path):
            # Check if the file has an image extension
            if os.path.splitext(filename)[1].lower() in image_extensions:
                file_path = os.path.join(save_path, filename)
                os.remove(file_path)
        print(f"Deleted images")

    # Get all image and json file names
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    json_files = [f for f in os.listdir(json_folder) if f.lower().endswith('.json')]
    if len(image_files) != len(json_files):
        print("Number of images and json files do not match!")
    else:
        for image_file in image_files:
            # Construct full image path
            image_path = os.path.join(images_folder, image_file)

            # Corresponding json file name (assuming they have the same base name)
            json_file_name = os.path.splitext(image_file)[0] + '.json'
            if json_file_name in json_files:
                json_path = os.path.join(json_folder, json_file_name)

                # Process the image and json
                process_image_and_json(image_path, json_path, save_path, name_format)

# Example usage
images_folder = '../../personal_data/Ruri_Dragon/raw'  # Update this path
json_folder = '../../personal_data/Ruri_Dragon/json_results'  # Update this path
save_path = "./panel_images_full_chapter"
name_format = "page_{:03}_panel_{:03}_bubble_{:03}{}"
nuke_option = True
process_all_images_and_jsons(images_folder, json_folder, save_path, name_format, nuke_option)


Deleted images
Processed and saved modified panel images for: 00.jpg
Processed and saved modified panel images for: 01.jpg
Processed and saved modified panel images for: 02.jpg
Processed and saved modified panel images for: 03.jpg
Processed and saved modified panel images for: 04.jpg
Processed and saved modified panel images for: 05.jpg
Processed and saved modified panel images for: 06.jpg
Processed and saved modified panel images for: 07.jpg
Processed and saved modified panel images for: 08.jpg
Processed and saved modified panel images for: 09.jpg
Processed and saved modified panel images for: 10.jpg
Processed and saved modified panel images for: 11.jpg
Processed and saved modified panel images for: 12.jpg
Processed and saved modified panel images for: 13.jpg
Processed and saved modified panel images for: 14.jpg
Processed and saved modified panel images for: 15.jpg
Processed and saved modified panel images for: 16.jpg
Processed and saved modified panel images for: 17.jpg
Processed and

## Final version which has both flag for panel view

In [35]:
import json
import os
from PIL import Image, ImageDraw


def read_coordinates(json_file_path: str):
    """Read essential text coordinates from the given JSON file."""
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return [
        text for text, is_essential in zip(data["texts"], data["is_essential_text"]) if is_essential
    ], data["panels"]  # Return text coordinates and panel coordinates

def process_full_page(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    # Extract the text coordinates
    text_coords, _ = read_coordinates(json_file_path)
    
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        # Create a draw object on the original image copy
        draw = ImageDraw.Draw(img)

        # Initialize the modified panel image path
        modified_panel_image_path = os.path.join(save_path, name_format)
        text_order = total_length_text

        # Save the modified panel image with the last computed bubble index
        img.save(modified_panel_image_path.format(image_name, 0, text_order, image_extension))

        # Loop through each set of coordinates in reverse order
        for box_index, box in enumerate(reversed(text_coords)):
            x1, y1, x2, y2 = map(int, box)  # Ensures all values are integers

            # Initialize max color and max value
            max_color = (0, 0, 0)  # Start with black
            max_value = 0

            # Iterate through the box's pixels
            for y in range(y1, y2):
                for x in range(x1, x2):
                    # Get the pixel color using absolute coordinates
                    color = img.getpixel((x, y))
                    color_value = sum(color[:3])  # Sum RGB values

                    # Update max color if the current pixel's value is higher
                    if color_value > max_value:
                        max_value = color_value
                        max_color = color

            # Fill the rectangle with the max color
            draw.rectangle(box, fill=max_color)

            # Save modified image path for the current chat bubble
            text_order = total_length_text - (box_index + 1)
            modified_image_path = modified_panel_image_path.format(image_name, 0, text_order, image_extension)

            # Instead of saving here, save it once at the end
            img.save(modified_image_path, format='JPEG')

    print(f"Processed and saved modified full page images for: {image_name_ext}")

def process_panel_view(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    text_coords, panel_coords = read_coordinates(json_file_path)
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, _ = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        # Create a copy of the image for drawing
        original_img = img.copy()

        # Loop through each panel
        for panel_index, panel in enumerate(panel_coords):
            box_index_last_frame = -1
            # Create a panel-specific image by cropping the original image
            panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
            panel_image = original_img.crop(panel_box)
            original_panel_image = panel_image.copy()

            # Create a draw object on the panel image copy
            draw = ImageDraw.Draw(panel_image)

            modified_panel_image_path = os.path.join(save_path, name_format)
            text_order = total_length_text

            satisfied_coor = []
            for box_index, box in enumerate(reversed(text_coords)):
                # Calculate the center point of the chatbox
                x_center = (box[0] + box[2]) // 2
                y_center = (box[1] + box[3]) // 2

                # Check if the center point is inside the panel
                if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                    satisfied_coor.append((box_index, box))
            satisfied_coor_len = len(satisfied_coor)
            # Loop through each set of coordinates in reverse order
            for satisfied_coor_index, [box_index, box] in enumerate(satisfied_coor):
                # Calculate the position of the box relative to the panel
                relative_box = (
                    box[0] - panel[0],
                    box[1] - panel[1],
                    box[2] - panel[0],
                    box[3] - panel[1]
                )

                # Get the coordinates of the relative box and convert to integers
                x1, y1, x2, y2 = map(int, relative_box)  # Ensures all values are integers

                # Initialize max color and max value
                max_color = (0, 0, 0)  # Start with black
                max_value = 0

                # Iterate through the box's pixels
                for y in range(y1, y2):
                    for x in range(x1, x2):
                        # Get the pixel color using absolute coordinates
                        color = original_img.getpixel((x + panel[0], y + panel[1]))
                        color_value = sum(color[:3])  # Sum RGB values

                        # Update max color if the current pixel's value is higher
                        if color_value > max_value:
                            max_value = color_value
                            max_color = color


                # Fill the rectangle with the max color
                draw.rectangle(relative_box, fill=max_color)

                # Save modified image path for the current chat bubble
                text_order = total_length_text - (box_index + 1)
                box_index_last_frame = max(text_order, box_index_last_frame)

                if satisfied_coor_len == 0 or satisfied_coor_index == satisfied_coor_len - 1:
                    # Save the modified panel image with the last computed bubble index
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, 0, '.jpg')
                else:
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg')

                # Instead of saving here, save it once at the end
                panel_image.save(modified_image_path, format='JPEG')
                # print(f"saving modified image: {modified_image_path}")

            text_order = box_index_last_frame + 1
            # Save the modified panel image with the last computed bubble index
            modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg')
            original_panel_image.save(modified_image_path, format='JPEG')
            # print(f"saving modified image: {modified_image_path}")

    print(f"Processed and saved modified panel view images for: {image_name_ext}")

def process_all_images_and_jsons(images_folder: str, json_folder: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}", nuke: bool = False, panel_view: bool = False):
    
    if nuke:
        try:
            image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}

            for filename in os.listdir(save_path):
                # Check if the file has an image extension
                if os.path.splitext(filename)[1].lower() in image_extensions:
                    file_path = os.path.join(save_path, filename)
                    os.remove(file_path)
            print(f"Deleted images")
        except FileNotFoundError as e:
            print(f"Error: {e}")

    # Get all image and json file names
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    json_files = [f for f in os.listdir(json_folder) if f.lower().endswith('.json')]
    if len(image_files) != len(json_files):
        print("Number of images and json files do not match!")
    else:
        for image_file in image_files[:5]:
            # Construct full image path
            image_path = os.path.join(images_folder, image_file)

            # Corresponding json file name (assuming they have the same base name)
            json_file_name = os.path.splitext(image_file)[0] + '.json'
            if json_file_name in json_files:
                json_path = os.path.join(json_folder, json_file_name)

                # Process the image and json based on panel_view flag
                if panel_view:
                    process_panel_view(image_path, json_path, save_path, name_format)
                else:
                    process_full_page(image_path, json_path, save_path, name_format)

# Example usage
images_folder = "../../personal_data/Ruri_Dragon/raw"  # Update this path
json_folder = "../../personal_data/Ruri_Dragon/json_results"  # Update this path
# save_path = "./panel_images_full_chapter"
save_path = "./panel_images"
name_format = "page_{:04}_panel_{:04}_bubble_{:04}{}"
nuke_option = True
panel_view_option = True  # Set this to True or False based on your needs

process_all_images_and_jsons(images_folder, json_folder, save_path, name_format, nuke_option, panel_view_option)


Deleted images
Processed and saved modified panel view images for: 00.jpg
Processed and saved modified panel view images for: 01.jpg
Processed and saved modified panel view images for: 02.jpg
Processed and saved modified panel view images for: 03.jpg
Processed and saved modified panel view images for: 04.jpg


# Images to Video

In [41]:
import cv2
import os
from glob import glob

# Define the directory where the modified images are stored
image_dir = './panel_images'  # Change this to your actual save path
output_video_path = './panel_images/ruri_dragon.mp4'  # Change the extension to .mp4
fps = 30  # Desired frames per second for the video
duration_per_image = 0.5  # Duration each image is displayed in seconds

# Calculate the total frames for each image based on desired display duration
frames_per_image = int(fps * duration_per_image)

# Get all the modified images sorted in reverse order
image_files = sorted(glob(os.path.join(image_dir, 'page_*_panel_*_bubble_*.jpg')), reverse=False)

# Check if there are any image files
if not image_files:
    print("No modified images found.")
else:
    # Get the dimensions of the first image to set the video size
    first_image = cv2.imread(image_files[0])
    height, width, _ = first_image.shape

    # Create a VideoWriter object for MP4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)
        for _ in range(frames_per_image):
            video_writer.write(img)  # Write the same image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_path}")

Video saved as: ./panel_images/ruri_dragon.mp4


The above code only works with images with same height/width. To fix it there is 2 approaches:

Option 1: Fixed Video Size by zooming smaller images to fit either height or width

Using Padding or just Zoom to fit either height or width

## Final Version

In [6]:
import cv2
import os
import numpy as np
from glob import glob

import re

def convert_to_video_name_format(name_format: str) -> str:
    # Replace anything between { and } with {}
    return re.sub(r'\{[^}]*\}', '{}', name_format)

def create_video_from_images(image_dir: str, output_video_path: str, name_format: str, use_padding: bool, fps: int = 30, duration_per_image: float = 0.5, reverse_order: bool = False):

    video_name_format = convert_to_video_name_format(name_format)

    output_video_name = os.path.join(output_video_path, f'video_Padding_{use_padding}.mp4')
    # Calculate the total frames for each image based on desired display duration
    frames_per_image = int(fps * duration_per_image)

    # Get all the modified images sorted in reverse order using the provided video_name_format
    image_files = sorted(glob(os.path.join(image_dir, video_name_format.format('*', '*', '*', '*'))), reverse=reverse_order)

    # Check if there are any image files
    if not image_files:
        print("No modified images found.")
        return

    # Find the largest dimensions among the images
    max_height = 0
    max_width = 0

    for image_file in image_files:
        img = cv2.imread(image_file)
        height, width, _ = img.shape
        max_height = max(max_height, height)
        max_width = max(max_width, width)

    # Create a VideoWriter object for MP4 with the maximum dimensions
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    video_writer = cv2.VideoWriter(output_video_name, fourcc, fps, (max_width, max_height))

    # Loop through the images and write them to the video
    for image_file in image_files:
        img = cv2.imread(image_file)

        # Create a black background image with max dimensions if padding is needed
        if use_padding:
            background = np.zeros((max_height, max_width, 3), dtype=np.uint8)

            # Get the dimensions of the current image
            height, width, _ = img.shape
            
            # Calculate the position to place the current image on the background
            x_offset = (max_width - width) // 2
            y_offset = (max_height - height) // 2
            
            # Place the current image on the background
            background[y_offset:y_offset + height, x_offset:x_offset + width] = img
        else:
            # For no padding, fit the image to max dimensions
            height, width, _ = img.shape
            
            # Calculate the scaling factor to fit the image to the maximum width or height
            scale = min(max_width / width, max_height / height)
            new_width = int(width * scale)
            new_height = int(height * scale)

            # Resize the image
            resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

            # Create a black background image with max dimensions
            background = np.zeros((max_height, max_width, 3), dtype=np.uint8)
            
            # Calculate the position to place the resized image on the background
            x_offset = (max_width - new_width) // 2
            y_offset = (max_height - new_height) // 2
            
            # Place the resized image on the background
            background[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_img

        for _ in range(frames_per_image):
            video_writer.write(background)  # Write the (padded) image multiple times for the duration

    # Release the video writer
    video_writer.release()

    print(f"Video saved as: {output_video_name}")

# Example usage
# image_dir = "./panel_images_full_chapter"
# image_dir = "./panel_images"
image_dir = "./full_image"
name_format = "page_{:03}_panel_{:03}_bubble_{:03}{}"

# Create video without padding
# create_video_from_images(image_dir, output_video_path_no_padding, name_format, use_padding=False)

# Create video with padding
create_video_from_images(image_dir=image_dir, output_video_path=image_dir, name_format=name_format, use_padding=True)

Video saved as: ./full_image\video_Padding_True.mp4


Option 2: Create multiple videos and then joins them later

# Show custom box on images to check

In [ ]:
import json
import os
from PIL import Image, ImageDraw

images_dir = '../../personal_data/Ruri_Dragon/raw/01.jpg'  # Update this path


# Extract the text coordinates
text_coords = [[655.2008056640625,
            95.861328125,
            754.2132568359375,
            171.38124084472656]]

# Open the image
with Image.open(images_dir) as img:
    draw = ImageDraw.Draw(img)

    # Draw red boxes for each set of coordinates
    for box in text_coords:
        # box format: [x1, y1, x2, y2]
        draw.rectangle(box, outline="red", width=5)

    img.show()

print(f"Saved modified image: {modified_image_path}")

Saved modified image: ./modified_01.jpg
